In [37]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("012").getOrCreate()

In [38]:
datos=spark.read.csv("comidaperros.csv",inferSchema=True,header=True)

In [39]:
datos.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Estropeado: double (nullable = true)



In [40]:
datos.show(100)

+---+---+----+---+----------+
|  A|  B|   C|  D|Estropeado|
+---+---+----+---+----------+
|  4|  2|12.0|  3|       1.0|
|  5|  6|12.0|  7|       1.0|
|  6|  2|13.0|  6|       1.0|
|  4|  2|12.0|  1|       1.0|
|  4|  2|12.0|  3|       1.0|
| 10|  3|13.0|  9|       1.0|
|  8|  5|14.0|  5|       1.0|
|  5|  8|12.0|  8|       1.0|
|  6|  5|12.0|  9|       1.0|
|  3|  3|12.0|  1|       1.0|
|  9|  8|11.0|  3|       1.0|
|  1| 10|12.0|  3|       1.0|
|  1|  5|13.0| 10|       1.0|
|  2| 10|12.0|  6|       1.0|
|  1| 10|11.0|  4|       1.0|
|  5|  3|12.0|  2|       1.0|
|  4|  9|11.0|  8|       1.0|
|  5|  1|11.0|  1|       1.0|
|  4|  9|12.0| 10|       1.0|
|  5|  8|10.0|  9|       1.0|
|  5|  7|11.0|  9|       1.0|
|  4| 10|13.0|  8|       1.0|
| 10|  5|12.0|  9|       1.0|
|  2|  4|13.0|  4|       1.0|
|  1|  4|13.0| 10|       1.0|
|  1|  8|12.0|  1|       1.0|
|  2| 10|13.0|  4|       1.0|
|  6|  2|12.0|  4|       1.0|
|  8|  2|13.0|  3|       1.0|
|  6|  4|12.0|  2|       1.0|
|  3|  2|1

In [41]:
from pyspark.ml.feature import VectorAssembler

In [42]:
constructor=VectorAssembler(inputCols=['A', 'B', 'C', 'D'],outputCol="features")

In [43]:
datos.columns

['A', 'B', 'C', 'D', 'Estropeado']

In [44]:
datosFinales=constructor.transform(datos)

In [45]:
datosFinales.show()

+---+---+----+---+----------+-------------------+
|  A|  B|   C|  D|Estropeado|           features|
+---+---+----+---+----------+-------------------+
|  4|  2|12.0|  3|       1.0| [4.0,2.0,12.0,3.0]|
|  5|  6|12.0|  7|       1.0| [5.0,6.0,12.0,7.0]|
|  6|  2|13.0|  6|       1.0| [6.0,2.0,13.0,6.0]|
|  4|  2|12.0|  1|       1.0| [4.0,2.0,12.0,1.0]|
|  4|  2|12.0|  3|       1.0| [4.0,2.0,12.0,3.0]|
| 10|  3|13.0|  9|       1.0|[10.0,3.0,13.0,9.0]|
|  8|  5|14.0|  5|       1.0| [8.0,5.0,14.0,5.0]|
|  5|  8|12.0|  8|       1.0| [5.0,8.0,12.0,8.0]|
|  6|  5|12.0|  9|       1.0| [6.0,5.0,12.0,9.0]|
|  3|  3|12.0|  1|       1.0| [3.0,3.0,12.0,1.0]|
|  9|  8|11.0|  3|       1.0| [9.0,8.0,11.0,3.0]|
|  1| 10|12.0|  3|       1.0|[1.0,10.0,12.0,3.0]|
|  1|  5|13.0| 10|       1.0|[1.0,5.0,13.0,10.0]|
|  2| 10|12.0|  6|       1.0|[2.0,10.0,12.0,6.0]|
|  1| 10|11.0|  4|       1.0|[1.0,10.0,11.0,4.0]|
|  5|  3|12.0|  2|       1.0| [5.0,3.0,12.0,2.0]|
|  4|  9|11.0|  8|       1.0| [4.0,9.0,11.0,8.0]|


In [46]:
datosEntrena,datosEvalua=datosFinales.randomSplit([0.7,0.3])

In [47]:
from pyspark.ml.classification import (DecisionTreeClassifier,
                                    RandomForestClassifier,
                                    GBTClassifier)

In [48]:
dt=DecisionTreeClassifier(labelCol="Estropeado")
rf=RandomForestClassifier(labelCol="Estropeado",numTrees=40)
gt=GBTClassifier(labelCol="Estropeado")

In [49]:
modelodt=dt.fit(datosEntrena)
modelorf=rf.fit(datosEntrena)
modelogt=gt.fit(datosEntrena)

In [50]:
modelorf.featureImportances,modelodt.featureImportances,modelogt.featureImportances

(SparseVector(4, {0: 0.0294, 1: 0.0181, 2: 0.9244, 3: 0.0281}),
 SparseVector(4, {0: 0.0109, 1: 0.0094, 2: 0.9675, 3: 0.0122}),
 SparseVector(4, {0: 0.063, 1: 0.0871, 2: 0.7937, 3: 0.0562}))

In [51]:
evaluadt=modelodt.transform(datosEvalua)
evaluarf=modelorf.transform(datosEvalua)
evaluagt=modelogt.transform(datosEvalua)

In [52]:
evaluarf.select("Estropeado","prediction").show()

+----------+----------+
|Estropeado|prediction|
+----------+----------+
|       1.0|       0.0|
|       1.0|       1.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       1.0|       1.0|
|       0.0|       0.0|
|       1.0|       1.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
+----------+----------+
only showing top 20 rows



In [53]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator

In [54]:
evaluadorB=BinaryClassificationEvaluator(labelCol="Estropeado")

In [55]:
evaluadorB.evaluate(evaluadt),evaluadorB.evaluate(evaluarf),evaluadorB.evaluate(evaluagt)

(0.9722644376899697, 0.992401215805471, 0.9732142857142857)

In [56]:
evaluadorM=MulticlassClassificationEvaluator(labelCol="Estropeado",metricName="accuracy")

In [57]:
evaluadorM.evaluate(evaluadt),evaluadorM.evaluate(evaluarf),evaluadorM.evaluate(evaluagt)

(0.9748427672955975, 0.9811320754716981, 0.9685534591194969)

In [58]:
nuevoModeloRf=rf.fit(datosFinales)

In [59]:
nuevoModeloRf.featureImportances

SparseVector(4, {0: 0.0239, 1: 0.0186, 2: 0.9332, 3: 0.0244})

In [62]:
datosFinales.select(datosFinales[2],datosFinales["Estropeado"]).show(200)

+----+----------+
|   C|Estropeado|
+----+----------+
|12.0|       1.0|
|12.0|       1.0|
|13.0|       1.0|
|12.0|       1.0|
|12.0|       1.0|
|13.0|       1.0|
|14.0|       1.0|
|12.0|       1.0|
|12.0|       1.0|
|12.0|       1.0|
|11.0|       1.0|
|12.0|       1.0|
|13.0|       1.0|
|12.0|       1.0|
|11.0|       1.0|
|12.0|       1.0|
|11.0|       1.0|
|11.0|       1.0|
|12.0|       1.0|
|10.0|       1.0|
|11.0|       1.0|
|13.0|       1.0|
|12.0|       1.0|
|13.0|       1.0|
|13.0|       1.0|
|12.0|       1.0|
|13.0|       1.0|
|12.0|       1.0|
|13.0|       1.0|
|12.0|       1.0|
|11.0|       1.0|
|12.0|       1.0|
|13.0|       1.0|
|12.0|       1.0|
|13.0|       1.0|
|12.0|       1.0|
|12.0|       1.0|
|12.0|       1.0|
|11.0|       1.0|
|11.0|       1.0|
|12.0|       1.0|
|11.0|       1.0|
|11.0|       1.0|
|11.0|       1.0|
|11.0|       1.0|
|11.0|       1.0|
|11.0|       1.0|
|13.0|       1.0|
|13.0|       1.0|
|12.0|       1.0|
|12.0|       1.0|
|14.0|       1.0|
|11.0|    